## KoBERT를 이용한 korNLI풀어보기(다중 클래스 분류)
KorNLI 데이터는 카카오 브레인에서 공개한 한국어 벤치마크 데이터셋입니다.
KorNLI 데이터셋의 깃허브: https://github.com/kakaobrain/KorNLUDatasets
NLI(Natural Language Inferencing) 는 두 개의 문장이 주어지고, 두 개의 문장이 수반 (entailment) 관계
인지, 모순 (contradiction) 관계인지, 중립 (neutral) 관계인지를 맞추는 문제입니다. 이번에는 한국어
NLI 데이터셋인 KorNLI 데이터셋을 가지고 두 개의 문장을 입력받아서 세 개 중 하나인 관계를 맞추는 다
중 클래스 분류 문제를 풀어봅시다. 그리고 BERT 가 두 개의 문장을 입력받을 때는 어떤 식으로 전처리를
진행하는지 보겠습니다.

In [1]:
!pip install transformers

###1. 데이터로드

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tf_keras.callbacks import EarlyStopping, ModelCheckpoint

훈련 데이터, 검증 데이터, 테스트 데이터로 사용할 NLI데이터를 로드합니다.

In [3]:
# 훈 련 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
# 검 증 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")
# 테 스 트 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7839dadb2e50>)

훈련 데이터로 사용할 두개의 파일과, 검증 데이터와 테스트 데이터 파일을 각각 데이터프레임으로 로드합니다.

In [4]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

훈련 데이터로 사용할 두개의 데이터 프레임을 학습을 위해 하나로 합쳐줍니다. 그리고 데이터를 섞어줍니다.

In [5]:
# 결 합 후 섞 기
# Pandas 2.0 이상에서 이 오류는 최근 버전의 Pandas 2.0 이상에서 DataFrame.append() 메서드를 사용해서 행을 추가할 수 있었지만, 이제는 완전히 제거되었어요.
train_data = pd.concat([train_snli, train_xnli], ignore_index=True)
train_data = train_data.sample(frac=1)

In [6]:
train_data.head(10)

,sentence1,sentence2,gold_label
11688,자전거를 탄 노인이 나무 꼭대기를 통해 태양을 걸러내며 숲이 우거진 시골길을 달리고...,한 노인이 자선 여행을 위해 자전거 도로에 있다.,neutral
649598,로마 또는 무어 출신인 공원의 후추 냄비 등대는 카르타헤나의 수비수들에 의해 정전되...,카르타헤나의 수비수들은 항상 등대가 켜졌는지 확인했다.,contradiction
536060,검은 티셔츠를 입은 여자가 에스컬레이터를 타고 올라가는 것 같다.,한 여자가 쇼핑몰 2층으로 가고 있다.,neutral
912796,이 전시회는 음악과 아일랜드 문화 사이의 중요한 코넥션을 설명하며 볼 가치가 있습니다.,그 전시회는 그들의 의도된 목적을 잘 수행하며 보여져야 한다.,entailment
379160,크레인 한 대와 노동자 3명이 타고 있는 트럭이 녹색 덮개가 달린 건설 프로젝트 앞...,세 명의 전기 기사가 건설현장에 전력을 연결하기 위해 일한다.,neutral
348832,사람들이 술집에 모여 있다.,사람들은 파티를 연다,neutral
342667,빨간 탱크톱을 쓴 소녀가 현관에 앉아 있다.,탱크톱을 입은 소녀가 태양을 즐기고 있다.,entailment
461343,두 젊은 여성이 대도시가 내려다보이는 커다란 창문 앞에 앉아 있다.,여자가 창문 앞에 앉아 있다.,entailment
102359,모자에 노란 줄무늬가 있는 어린 소년은 눈 속에 있다.,그 어린 소년은 노란 모자를 가지고 있다,entailment
802135,훈련 전략은 우리의 임무에 대한 차별에 기초한 옹호의 중심성을 강조한다.,차별은 여전히 직장에서 이슈가 되고 있다.,entailment


In [7]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [8]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


훈련 데이터, 검증 데이터, 테스트 데이터에 중복 샘플이나 결측값이 있는 샘플이 있다면 제거해줍니다.
이는 dropn_na_and_duplciates 라는 함수를 사용하여 적용합니다.

In [9]:
def drop_na_and_duplicates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop = True)
  return df

In [10]:
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

각 데이터의 샘플 수는 다음과 같습니다.

In [11]:
print("훈련 데이터의 개수: ", len(train_data))
print("검증 데이터의 개수: ", len(val_data))
print("테스트 데이터의 개수: ", len(test_data))

훈련 데이터의 개수:  941814
검증 데이터의 개수:  2490
테스트 데이터의 개수:  5010


###2.BERT의 입력
토큰화를 위해 BERT 토크나이저를 로드 합니다.

In [12]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

훈련 데이터의 첫번째 샘플을 선택하여 토크나이저의 전처리 결과를 봅시다. 여기서는 최대 길이를 128로 정했습니다.

In [13]:
max_seq_len = 144

sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print('문장1: ', sent1)
print('문장2: ', sent2)

문장1:  자전거를 탄 노인이 나무 꼭대기를 통해 태양을 걸러내며 숲이 우거진 시골길을 달리고 있다.
문장2:  한 노인이 자선 여행을 위해 자전거 도로에 있다.


encoder_plus는 정수 인코딩 결과인 input_ids, 두개의 문장을 구분하기 위한 token_type_ids, 그리고 어텐션 마스크에 해당하는 attention_mask를 자동으로 생성하여 리턴해 줍니다. 두 개의 문장을 BERT에 넣을 때는 한번에 넣어야 하기 때문에 encoder_plus의 입력으로 sent1과 sent2를 이어서 넣어줍니다.

In [14]:

# pad_to_max_length=True 가 padding='max_length'로 바뀜
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, truncation=True, padding='max_length',)



전처리 결과를 봅시다. encode_plus 가 리턴한 encoding_result 로부터 input_ids 의 값을 호출하면 정
수 인코딩 결과를 확인할 수 있습니다.

In [15]:
print(encoding_result['input_ids'])

[2, 5655, 2138, 1763, 4662, 2052, 4506, 15250, 2138, 3644, 5275, 2069, 17023, 2369, 2307, 1305, 2052, 29173, 7535, 2454, 2069, 4214, 2088, 1513, 2062, 18, 3, 1891, 4662, 2052, 13895, 4062, 2069, 3627, 5655, 4475, 2170, 1513, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


맨 앞은 정수 2 가 있고 중간에 정수 3 이 두 번 등장합니다. 그리고 두번째 3 뒤에는 0 이 이어집니다. 2 는
[CLS], 3 은 [SEP], 0 은 [PAD] 를 의미하므로 현재 정수 인코딩 결과는 다음과 같은 구성으로 이루어져 있
습니다.

• [CLS] 문장 1 [SEP] 문장 2 [SEP] [PAD] ... 중략 ... [PAD]

맨 처음에 [CLS] 가 등장하고 문장 1 이 끝날 때 [SEP] 가 등장하며, 문장 2 가 끝날 때 [SEP] 가 재등장한 후
에 패딩 토큰이 문장의 최대 길이까지 이어지는 구성입니다. 이번에는 문장을 구분하기 위한 세그먼트 인
코딩에 해당하는 token_type_ids 의 값을 보겠습니다.

In [16]:
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


문장 1 의 구간까지는 0 이 반복되다가 문장 2 의 구간부터는 1 이 반복된 후 문장 2 가 끝나는 구간. 즉,
[PAD] 가 시작되는 구간에서는 다시 0 이 반복됩니다. 앞의 실습에서 문장 구분이 없었던 문제들과는 달
리 token_type_ids 를 통해서 두 개의 문장을 구분하고 있음을 알 수 있습니다.

In [17]:
print(encoding_result['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


어텐션 마스크는 앞에서 풀어왔던 문제들과 마찬가지로 실제 단어 토큰의 위치에는 1 이 반복되다가 패
딩 토큰인 [PAD] 가 등장하는 구간부터는 0 이 반복됩니다. 훈련 데이터의 첫번째 샘플을 가지고 임의로
진행했던 전처리를 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 모두 진행해봅시다. 이 과정은
convert_examples_to_features 함수를 통해서 진행합니다.

In [18]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length = max_seq_len, padding='max_length', truncation=True) # pad_to_max_length=True 가 padding='max_length'로 바뀜

    # Ensure the lengths are consistent
    if len(encoding_result['input_ids']) == max_seq_len:
      input_ids.append(encoding_result['input_ids'])
      attention_masks.append(encoding_result['attention_mask'])
      token_type_ids.append(encoding_result['token_type_ids'])

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

In [19]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 2846/941814 [00:01<06:44, 2319.25it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|▏         | 13437/941814 [00:05<06:39, 2323.17it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have

훈련 데이터의 첫번째 샘플에 대한 정수 인코딩, 어텐션 마스크, 세그먼트 인코딩, 정수 인코딩을 기존의
문자열로 복원한 결과는 다음과 같습니다.

In [20]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]
print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

단 어 에 대 한 정 수 인 코 딩 : [    2  5655  2138  1763  4662  2052  4506 15250  2138  3644  5275  2069
 17023  2369  2307  1305  2052 29173  7535  2454  2069  4214  2088  1513
  2062    18     3  1891  4662  2052 13895  4062  2069  3627  5655  4475
  2170  1513  2062    18     3     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0

검증 데이터에 대해서도 전처리를 진행해봅시다.

In [21]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 2490/2490 [00:01<00:00, 2135.23it/s]


검증 데이터의 첫번째 샘플에 대해서 전처리를 진행한 결과는 다음과 같습니다.

In [22]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]
print('단 어 에 대 한 정 수 인 코 딩 :',input_id)
print('어 텐 션 마 스 크 :',attention_mask)
print('세 그 먼 트 인 코 딩 :',token_type_id)
print('각 인 코 딩 의 길 이 :', len(input_id))
print('정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

단 어 에 대 한 정 수 인 코 딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0

테스트 데이터에 대해서 전처리를 진행해봅시다.

In [23]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'],max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 5010/5010 [00:02<00:00, 2166.00it/s]


contradiction, entailment, neutral 과 같이 문자열로 구성된 레이블에 대해서도 정수 인코딩을 진행합니
다.

In [24]:
# .tolist()는 NumPy 배열이나 Pandas 객체를 일반적인 Python 리스트로 변환할 때 사용
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고 유 한 정 수 로 변 환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레 이 블 과 정 수 :', label_idx)

각 레 이 블 과 정 수 : {np.str_('contradiction'): np.int64(0), np.str_('entailment'): np.int64(1), np.str_('neutral'): np.int64(2)}


레이블을 정수 인코딩을 하기 전과 후를 훈련 데이터에 대해서 상위 5 개를 출력하여 비교해봅시다.

In [25]:
print('변 환 전 :', train_label[:5])
print('변 환 후 :',y_train[:5])

변 환 전 : ['neutral', 'contradiction', 'neutral', 'entailment', 'neutral']
변 환 후 : [2 0 2 1 2]


###3. BERT를 이용한 다중 클래스 분류 모델
다중 클래스 분류 문제를 수행하는 모델을 만들어봅시다. BERT 의 CLS 토큰의 출력에 소프트맥스 함수를
사용하는 출력층을 연결합니다. BERT 의 출력을 outputs 로 리턴받았다면, outputs[1] 을 통해 CLS 토큰위치의 벡터를 사용할 수 있습니다.

In [26]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer = tf.keras.
                                            initializers.TruncatedNormal
                                             (0.02),
                                            activation='softmax',
                                            name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)
    return prediction

In [27]:
model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [28]:
early_stopping = EarlyStopping(monitor="val_accuracy", min_delta=0.001,patience=2)
model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data = (X_val, y_val), callbacks = [early_stopping])

Epoch 1/2
29432/29432 [==============================] - 3149s 106ms/step - loss: 0.5330 - accuracy: 0.7881 - val_loss: 0.5917 - val_accuracy: 0.7574
Epoch 2/2
29432/29432 [==============================] - 3084s 105ms/step - loss: 0.4354 - accuracy: 0.8325 - val_loss: 0.5454 - val_accuracy: 0.7876


In [29]:
print("/n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))

5/5 [==============================] - 4s 823ms/step - loss: 0.5437 - accuracy: 0.7870
/n 테스트 정확도: 0.7870


###4. TFBertForSequenceClassification
만약 transformers에서 제공하는 구현체인 TFBertForSequenceClassification을 사용했다면 다음과 같이 코드를 사용할 수 있습니다.

In [34]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 3, from_pt = True)
model.compile(optimizer = optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
